In [1]:
import numpy as np
import pandas as pd

import keras
import tensorflow as tf


from keras.preprocessing import image
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten,BatchNormalization,Activation
from keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import Adam, SGD, RMSprop
from tensorflow.python.keras import backend as K

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2

In [4]:
train_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/train/'
test_dir='/kaggle/input/siim-isic-melanoma-classification/jpeg/test/'

In [5]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')

In [6]:
train['target'].value_counts()

0    32542
1      584
Name: target, dtype: int64

In [7]:
train_samples = train.copy()
train_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33126 entries, 0 to 33125
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   image_name                     33126 non-null  object 
 1   patient_id                     33126 non-null  object 
 2   sex                            33061 non-null  object 
 3   age_approx                     33058 non-null  float64
 4   anatom_site_general_challenge  32599 non-null  object 
 5   diagnosis                      33126 non-null  object 
 6   benign_malignant               33126 non-null  object 
 7   target                         33126 non-null  int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 2.0+ MB


In [8]:
train_labels = []
train_images =[]

for i in range(train.shape[0]):
    print("Index: {}", i)
    train_images.append(train_dir+train_samples['image_name'].iloc[i]+'.jpg')
    train_labels.append(train_samples['target'].iloc[i])

Index: {} 0
Index: {} 1
Index: {} 2
Index: {} 3
Index: {} 4
Index: {} 5
Index: {} 6
Index: {} 7
Index: {} 8
Index: {} 9
Index: {} 10
Index: {} 11
Index: {} 12
Index: {} 13
Index: {} 14
Index: {} 15
Index: {} 16
Index: {} 17
Index: {} 18
Index: {} 19
Index: {} 20
Index: {} 21
Index: {} 22
Index: {} 23
Index: {} 24
Index: {} 25
Index: {} 26
Index: {} 27
Index: {} 28
Index: {} 29
Index: {} 30
Index: {} 31
Index: {} 32
Index: {} 33
Index: {} 34
Index: {} 35
Index: {} 36
Index: {} 37
Index: {} 38
Index: {} 39
Index: {} 40
Index: {} 41
Index: {} 42
Index: {} 43
Index: {} 44
Index: {} 45
Index: {} 46
Index: {} 47
Index: {} 48
Index: {} 49
Index: {} 50
Index: {} 51
Index: {} 52
Index: {} 53
Index: {} 54
Index: {} 55
Index: {} 56
Index: {} 57
Index: {} 58
Index: {} 59
Index: {} 60
Index: {} 61
Index: {} 62
Index: {} 63
Index: {} 64
Index: {} 65
Index: {} 66
Index: {} 67
Index: {} 68
Index: {} 69
Index: {} 70
Index: {} 71
Index: {} 72
Index: {} 73
Index: {} 74
Index: {} 75
Index: {} 76
Index: {}

Index: {} 792
Index: {} 793
Index: {} 794
Index: {} 795
Index: {} 796
Index: {} 797
Index: {} 798
Index: {} 799
Index: {} 800
Index: {} 801
Index: {} 802
Index: {} 803
Index: {} 804
Index: {} 805
Index: {} 806
Index: {} 807
Index: {} 808
Index: {} 809
Index: {} 810
Index: {} 811
Index: {} 812
Index: {} 813
Index: {} 814
Index: {} 815
Index: {} 816
Index: {} 817
Index: {} 818
Index: {} 819
Index: {} 820
Index: {} 821
Index: {} 822
Index: {} 823
Index: {} 824
Index: {} 825
Index: {} 826
Index: {} 827
Index: {} 828
Index: {} 829
Index: {} 830
Index: {} 831
Index: {} 832
Index: {} 833
Index: {} 834
Index: {} 835
Index: {} 836
Index: {} 837
Index: {} 838
Index: {} 839
Index: {} 840
Index: {} 841
Index: {} 842
Index: {} 843
Index: {} 844
Index: {} 845
Index: {} 846
Index: {} 847
Index: {} 848
Index: {} 849
Index: {} 850
Index: {} 851
Index: {} 852
Index: {} 853
Index: {} 854
Index: {} 855
Index: {} 856
Index: {} 857
Index: {} 858
Index: {} 859
Index: {} 860
Index: {} 861
Index: {} 862
Index:

Index: {} 1534
Index: {} 1535
Index: {} 1536
Index: {} 1537
Index: {} 1538
Index: {} 1539
Index: {} 1540
Index: {} 1541
Index: {} 1542
Index: {} 1543
Index: {} 1544
Index: {} 1545
Index: {} 1546
Index: {} 1547
Index: {} 1548
Index: {} 1549
Index: {} 1550
Index: {} 1551
Index: {} 1552
Index: {} 1553
Index: {} 1554
Index: {} 1555
Index: {} 1556
Index: {} 1557
Index: {} 1558
Index: {} 1559
Index: {} 1560
Index: {} 1561
Index: {} 1562
Index: {} 1563
Index: {} 1564
Index: {} 1565
Index: {} 1566
Index: {} 1567
Index: {} 1568
Index: {} 1569
Index: {} 1570
Index: {} 1571
Index: {} 1572
Index: {} 1573
Index: {} 1574
Index: {} 1575
Index: {} 1576
Index: {} 1577
Index: {} 1578
Index: {} 1579
Index: {} 1580
Index: {} 1581
Index: {} 1582
Index: {} 1583
Index: {} 1584
Index: {} 1585
Index: {} 1586
Index: {} 1587
Index: {} 1588
Index: {} 1589
Index: {} 1590
Index: {} 1591
Index: {} 1592
Index: {} 1593
Index: {} 1594
Index: {} 1595
Index: {} 1596
Index: {} 1597
Index: {} 1598
Index: {} 1599
Index: {} 

Index: {} 2221
Index: {} 2222
Index: {} 2223
Index: {} 2224
Index: {} 2225
Index: {} 2226
Index: {} 2227
Index: {} 2228
Index: {} 2229
Index: {} 2230
Index: {} 2231
Index: {} 2232
Index: {} 2233
Index: {} 2234
Index: {} 2235
Index: {} 2236
Index: {} 2237
Index: {} 2238
Index: {} 2239
Index: {} 2240
Index: {} 2241
Index: {} 2242
Index: {} 2243
Index: {} 2244
Index: {} 2245
Index: {} 2246
Index: {} 2247
Index: {} 2248
Index: {} 2249
Index: {} 2250
Index: {} 2251
Index: {} 2252
Index: {} 2253
Index: {} 2254
Index: {} 2255
Index: {} 2256
Index: {} 2257
Index: {} 2258
Index: {} 2259
Index: {} 2260
Index: {} 2261
Index: {} 2262
Index: {} 2263
Index: {} 2264
Index: {} 2265
Index: {} 2266
Index: {} 2267
Index: {} 2268
Index: {} 2269
Index: {} 2270
Index: {} 2271
Index: {} 2272
Index: {} 2273
Index: {} 2274
Index: {} 2275
Index: {} 2276
Index: {} 2277
Index: {} 2278
Index: {} 2279
Index: {} 2280
Index: {} 2281
Index: {} 2282
Index: {} 2283
Index: {} 2284
Index: {} 2285
Index: {} 2286
Index: {} 

Index: {} 3056
Index: {} 3057
Index: {} 3058
Index: {} 3059
Index: {} 3060
Index: {} 3061
Index: {} 3062
Index: {} 3063
Index: {} 3064
Index: {} 3065
Index: {} 3066
Index: {} 3067
Index: {} 3068
Index: {} 3069
Index: {} 3070
Index: {} 3071
Index: {} 3072
Index: {} 3073
Index: {} 3074
Index: {} 3075
Index: {} 3076
Index: {} 3077
Index: {} 3078
Index: {} 3079
Index: {} 3080
Index: {} 3081
Index: {} 3082
Index: {} 3083
Index: {} 3084
Index: {} 3085
Index: {} 3086
Index: {} 3087
Index: {} 3088
Index: {} 3089
Index: {} 3090
Index: {} 3091
Index: {} 3092
Index: {} 3093
Index: {} 3094
Index: {} 3095
Index: {} 3096
Index: {} 3097
Index: {} 3098
Index: {} 3099
Index: {} 3100
Index: {} 3101
Index: {} 3102
Index: {} 3103
Index: {} 3104
Index: {} 3105
Index: {} 3106
Index: {} 3107
Index: {} 3108
Index: {} 3109
Index: {} 3110
Index: {} 3111
Index: {} 3112
Index: {} 3113
Index: {} 3114
Index: {} 3115
Index: {} 3116
Index: {} 3117
Index: {} 3118
Index: {} 3119
Index: {} 3120
Index: {} 3121
Index: {} 

...

Index: {} 29003
Index: {} 29004
Index: {} 29005
Index: {} 29006
Index: {} 29007
Index: {} 29008
Index: {} 29009
Index: {} 29010
Index: {} 29011
Index: {} 29012
Index: {} 29013
Index: {} 29014
Index: {} 29015
Index: {} 29016
Index: {} 29017
Index: {} 29018
Index: {} 29019
Index: {} 29020
Index: {} 29021
Index: {} 29022
Index: {} 29023
Index: {} 29024
Index: {} 29025
Index: {} 29026
Index: {} 29027
Index: {} 29028
Index: {} 29029
Index: {} 29030
Index: {} 29031
Index: {} 29032
Index: {} 29033
Index: {} 29034
Index: {} 29035
Index: {} 29036
Index: {} 29037
Index: {} 29038
Index: {} 29039
Index: {} 29040
Index: {} 29041
Index: {} 29042
Index: {} 29043
Index: {} 29044
Index: {} 29045
Index: {} 29046
Index: {} 29047
Index: {} 29048
Index: {} 29049
Index: {} 29050
Index: {} 29051
Index: {} 29052
Index: {} 29053
Index: {} 29054
Index: {} 29055
Index: {} 29056
Index: {} 29057
Index: {} 29058
Index: {} 29059
Index: {} 29060
Index: {} 29061
Index: {} 29062
Index: {} 29063
Index: {} 29064
Index: {

Index: {} 29962
Index: {} 29963
Index: {} 29964
Index: {} 29965
Index: {} 29966
Index: {} 29967
Index: {} 29968
Index: {} 29969
Index: {} 29970
Index: {} 29971
Index: {} 29972
Index: {} 29973
Index: {} 29974
Index: {} 29975
Index: {} 29976
Index: {} 29977
Index: {} 29978
Index: {} 29979
Index: {} 29980
Index: {} 29981
Index: {} 29982
Index: {} 29983
Index: {} 29984
Index: {} 29985
Index: {} 29986
Index: {} 29987
Index: {} 29988
Index: {} 29989
Index: {} 29990
Index: {} 29991
Index: {} 29992
Index: {} 29993
Index: {} 29994
Index: {} 29995
Index: {} 29996
Index: {} 29997
Index: {} 29998
Index: {} 29999
Index: {} 30000
Index: {} 30001
Index: {} 30002
Index: {} 30003
Index: {} 30004
Index: {} 30005
Index: {} 30006
Index: {} 30007
Index: {} 30008
Index: {} 30009
Index: {} 30010
Index: {} 30011
Index: {} 30012
Index: {} 30013
Index: {} 30014
Index: {} 30015
Index: {} 30016
Index: {} 30017
Index: {} 30018
Index: {} 30019
Index: {} 30020
Index: {} 30021
Index: {} 30022
Index: {} 30023
Index: {

Index: {} 30890
Index: {} 30891
Index: {} 30892
Index: {} 30893
Index: {} 30894
Index: {} 30895
Index: {} 30896
Index: {} 30897
Index: {} 30898
Index: {} 30899
Index: {} 30900
Index: {} 30901
Index: {} 30902
Index: {} 30903
Index: {} 30904
Index: {} 30905
Index: {} 30906
Index: {} 30907
Index: {} 30908
Index: {} 30909
Index: {} 30910
Index: {} 30911
Index: {} 30912
Index: {} 30913
Index: {} 30914
Index: {} 30915
Index: {} 30916
Index: {} 30917
Index: {} 30918
Index: {} 30919
Index: {} 30920
Index: {} 30921
Index: {} 30922
Index: {} 30923
Index: {} 30924
Index: {} 30925
Index: {} 30926
Index: {} 30927
Index: {} 30928
Index: {} 30929
Index: {} 30930
Index: {} 30931
Index: {} 30932
Index: {} 30933
Index: {} 30934
Index: {} 30935
Index: {} 30936
Index: {} 30937
Index: {} 30938
Index: {} 30939
Index: {} 30940
Index: {} 30941
Index: {} 30942
Index: {} 30943
Index: {} 30944
Index: {} 30945
Index: {} 30946
Index: {} 30947
Index: {} 30948
Index: {} 30949
Index: {} 30950
Index: {} 30951
Index: {

Index: {} 31711
Index: {} 31712
Index: {} 31713
Index: {} 31714
Index: {} 31715
Index: {} 31716
Index: {} 31717
Index: {} 31718
Index: {} 31719
Index: {} 31720
Index: {} 31721
Index: {} 31722
Index: {} 31723
Index: {} 31724
Index: {} 31725
Index: {} 31726
Index: {} 31727
Index: {} 31728
Index: {} 31729
Index: {} 31730
Index: {} 31731
Index: {} 31732
Index: {} 31733
Index: {} 31734
Index: {} 31735
Index: {} 31736
Index: {} 31737
Index: {} 31738
Index: {} 31739
Index: {} 31740
Index: {} 31741
Index: {} 31742
Index: {} 31743
Index: {} 31744
Index: {} 31745
Index: {} 31746
Index: {} 31747
Index: {} 31748
Index: {} 31749
Index: {} 31750
Index: {} 31751
Index: {} 31752
Index: {} 31753
Index: {} 31754
Index: {} 31755
Index: {} 31756
Index: {} 31757
Index: {} 31758
Index: {} 31759
Index: {} 31760
Index: {} 31761
Index: {} 31762
Index: {} 31763
Index: {} 31764
Index: {} 31765
Index: {} 31766
Index: {} 31767
Index: {} 31768
Index: {} 31769
Index: {} 31770
Index: {} 31771
Index: {} 31772
Index: {

Index: {} 32553
Index: {} 32554
Index: {} 32555
Index: {} 32556
Index: {} 32557
Index: {} 32558
Index: {} 32559
Index: {} 32560
Index: {} 32561
Index: {} 32562
Index: {} 32563
Index: {} 32564
Index: {} 32565
Index: {} 32566
Index: {} 32567
Index: {} 32568
Index: {} 32569
Index: {} 32570
Index: {} 32571
Index: {} 32572
Index: {} 32573
Index: {} 32574
Index: {} 32575
Index: {} 32576
Index: {} 32577
Index: {} 32578
Index: {} 32579
Index: {} 32580
Index: {} 32581
Index: {} 32582
Index: {} 32583
Index: {} 32584
Index: {} 32585
Index: {} 32586
Index: {} 32587
Index: {} 32588
Index: {} 32589
Index: {} 32590
Index: {} 32591
Index: {} 32592
Index: {} 32593
Index: {} 32594
Index: {} 32595
Index: {} 32596
Index: {} 32597
Index: {} 32598
Index: {} 32599
Index: {} 32600
Index: {} 32601
Index: {} 32602
Index: {} 32603
Index: {} 32604
Index: {} 32605
Index: {} 32606
Index: {} 32607
Index: {} 32608
Index: {} 32609
Index: {} 32610
Index: {} 32611
Index: {} 32612
Index: {} 32613
Index: {} 32614
Index: {

In [9]:
test_images =[]
for i in range(test.shape[0]):
    test_images.append(test_dir+test['image_name'].iloc[i]+'.jpg')

df_test = pd.DataFrame(test_images)
df_test.columns = ['images']

In [10]:
print(train_images)

['/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_2637011.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0015719.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0052212.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0068279.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0074268.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0074311.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0074542.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0075663.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0075914.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0076262.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0076545.jpg', '/kaggle/input/siim-isic-melanoma-classification/jpeg/train/ISIC_0076742.jpg', '/kaggle/input/siim-isic-melanoma-classification/jp

In [11]:
df_train = pd.DataFrame(train_images)


In [12]:
df_train.columns =['images']

In [13]:
df_train['target'] = train_labels

In [14]:
print(df_train)

                                                  images  target
0      /kaggle/input/siim-isic-melanoma-classificatio...       0
1      /kaggle/input/siim-isic-melanoma-classificatio...       0
2      /kaggle/input/siim-isic-melanoma-classificatio...       0
3      /kaggle/input/siim-isic-melanoma-classificatio...       0
4      /kaggle/input/siim-isic-melanoma-classificatio...       0
...                                                  ...     ...
33121  /kaggle/input/siim-isic-melanoma-classificatio...       0
33122  /kaggle/input/siim-isic-melanoma-classificatio...       0
33123  /kaggle/input/siim-isic-melanoma-classificatio...       0
33124  /kaggle/input/siim-isic-melanoma-classificatio...       0
33125  /kaggle/input/siim-isic-melanoma-classificatio...       0

[33126 rows x 2 columns]


In [15]:
# Splitting the train data further into train and validation sets
X_train, X_val, y_train,y_val = train_test_split(df_train['images'],df_train['target'],test_size=0.2,random_state=0)

train = pd.DataFrame(X_train)
train.columns = ['images']
train['target']=y_train

validation = pd.DataFrame(X_val)
validation.columns = ['images']
validation['target']=y_val

In [16]:
def get_predictions(model,sub_df):
    target=[]
    for path in df_test['images']:
        img=cv2.imread(str(path))
        img = cv2.resize(img, (224,224))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)/255.
        img=np.reshape(img,(1,224,224,3))
        prediction=model.predict(img)
        target.append(prediction[0][0])
    
    sub_df['target']=target
    return sub_df

In [17]:
train_datagen = ImageDataGenerator(preprocess_input,rescale=1./255,rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocess_input,rescale=1./255)

image_size = 224

train_generator = train_datagen.flow_from_dataframe(
                    train,
                    x_col='images',
                    y_col ='target',
                    target_size=(image_size,image_size),
                    batch_size=8,
                    shuffle=True,
                    class_mode='raw')

validation_generator = val_datagen.flow_from_dataframe(
                    validation,
                    x_col='images',
                    y_col ='target',
                    target_size=(image_size,image_size),
                    batch_size=8,
                    shuffle=False,
                    class_mode='raw')

Found 26500 validated image filenames.
Found 6626 validated image filenames.


In [18]:
def vgg16_model(num_classes=None):
    model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
    x = Flatten()(model.output)
    output = Dense(1,activation='sigmoid')(x)
    model = Model(model.input,output)
    
    return model

In [19]:
def focal_loss(alpha=0.25,gamma=2.0):
    def focal_crossentropy(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        
        y_pred = K.clip(y_pred, K.epsilon(), 1.- K.epsilon())
        p_t = (y_true*y_pred) + ((1-y_true)*(1-y_pred))
        
        alpha_factor = 1
        modulating_factor = 1

        alpha_factor = y_true*alpha + ((1-alpha)*(1-y_true))
        modulating_factor = K.pow((1-p_t), gamma)

        # compute the final loss and return
        return K.mean(alpha_factor*modulating_factor*bce, axis=-1)
    return focal_crossentropy

In [20]:

# Defining the optimizer and compiling the model
opt = Adam(lr=1e-5)
vgg_conv = vgg16_model(1)
vgg_conv.compile(loss=focal_loss(),optimizer=opt,metrics=[keras.metrics.AUC()])

58892288/58889256 [==============================] - 1s 0us/step


In [21]:
# Denining the num of epochs, batch_size and steps for training and validation
nb_epochs = 5
batch_size = 16
nb_train_steps = train.shape[0]//batch_size  # // rounds off the result of division
nb_validation_steps = validation.shape[0]//batch_size
print("Number of training and validation steps are {} and {}".format(nb_train_steps,nb_validation_steps))

Number of training and validation steps are 207 and 51


In [22]:
# Fitting the model
vgg_conv.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_steps,
    epochs=nb_epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_steps)

/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/5


NotFoundError: {{function_node __inference_train_function_11966}} No registered 'PyFunc' OpKernel for 'CPU' devices compatible with node {{node PyFunc}}
	.  Registered:  <no registered kernels>

	 [[PyFunc]]
	 [[MultiDeviceIteratorGetNextFromShard]]
	 [[RemoteCall]]
	 [[IteratorGetNextAsOptional]]

In [ ]:
# Getting the predictions for test data

sub_vgg16 = test.copy()
sub_vgg16 = get_predictions(vgg_conv,sub_vgg16)
sub_vgg16.head()

In [ ]:
print(crash)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="test")

In [ ]:
help(RandomSearch)